In [15]:
import requests;


In [18]:
with open(r"pokemon.txt","r") as pk: #extracting the given names
    content = pk.readlines() 
    

In [ ]:
my_dict={} #dictionary for characterstics of every pokemon given

In [19]:
for i in range(0,50): #50 is the length
    name_id = content[i].strip() #removes \n
    r1=requests.get(f"https://pokeapi.co/api/v2/pokemon/{name_id}/") #requesting pokeapi for the given ID/name
    name=r1.json()["name"]
    idno=r1.json()["id"]
    type_=[]
    abilities=[]
    for ability in r1.json()["abilities"]: #looping for various abilities
            abilities.append(ability['ability']['name'])
    for types in r1.json()["types"]:
            type_.append(types['type']['name']) #looping for type(s)
    r2=requests.get(f"https://pokeapi.co/api/v2/pokemon-species/{name_id}/")#mythical and legendary parameters are given in these endpoint
    legendary=r2.json()["is_legendary"]
    mythical=r2.json()["is_mythical"]
    temp_dict={"id":idno,"is_legendary":legendary,"is_mythical":mythical,"abilities":abilities,"types":type_}
    my_dict[name]=temp_dict

In [25]:
damage_array={   #dictionary which will finally be the output
    "attackers": {},
    "defenders": {}
}
pokemon_types = [ '''stores in the format attackers->attackertype->{defender:multiplier}
                        or defenders->defendertype->{attacker:multiplier}'''
    "normal",
    "fire",
    "water",
    "electric",
    "grass",
    "ice",
    "fighting",
    "poison",
    "ground",
    "flying",
    "psychic",
    "bug",
    "rock",
    "ghost",
    "dragon",
    "dark",
    "steel",
    "fairy"
]
for j in range(1,19):#looping over the 18 types of pokemon
    r3=requests.get(f"https://pokeapi.co/api/v2/type/{j}/")
    name=r3.json()['name']
    for k in (r3.json()['damage_relations']):
            temp_attacker={}    #this gets updated when the pokemon defends in the format {nameofattacker:multiplier}  
            temp_defender={} #this gets updated when the pokemon attacks in the format {nameofdefender:multiplier}
            for ele1 in r3.json()['damage_relations']["double_damage_from"]: 
                        temp_defender[ele1["name"]]=2 #if it is defending and multiplier is 2
            for ele1 in r3.json()['damage_relations']["double_damage_to"]:
                        temp_attacker[ele1["name"]]=2 #if it is attacking and multiplier is 2 
            for ele1 in r3.json()['damage_relations']["half_damage_from"]:
                        temp_defender[ele1["name"]]=0.5 #
            for ele1 in r3.json()['damage_relations']["half_damage_to"]:
                        temp_attacker[ele1["name"]]=0.5
            for ele1 in r3.json()['damage_relations']["no_damage_from"]:
                        temp_defender[ele1["name"]]=0
            for ele1 in r3.json()['damage_relations']["no_damage_to"]:
                        temp_attacker[ele1["name"]]=0
    if (temp_attacker):    #non empty dict
        for ele in pokemon_types: #checking for neutral types
                    if ele not in temp_attacker.keys():
                            temp_attacker[ele]=1
        damage_array["attackers"].update({name:temp_attacker})
    if (temp_defender):
            for ele in pokemon_types:
                    if ele not in temp_defender.keys():
                            temp_defender[ele]=1
            damage_array["defenders"].update({name:temp_defender})



In [ ]:
from flask import Flask, request, jsonify 
app = Flask(__name__)

@app.route("/")
def damage_endpoint():
    attacker = request.args.get("attacker")
    defender = request.args.get("defender")
    
    if not attacker and not defender: # Both missing
        return jsonify({"error": "Provide attacker and defender"}), 400
    
    elif not attacker: #Only defender given
        try:
            data = damage_array["defenders"][defender]
            return jsonify({
                "defender": defender,
                "data": data
            })
        except KeyError:
            return jsonify({"error": f"No data for defender {defender}"}), 404
    
    elif not defender: # Only attacker given
        try:
            data = damage_array["attackers"][attacker]
            return jsonify({
                "attacker": attacker,
                "data": data
            })
        except KeyError:
            return jsonify({"error": f"No data for attacker {attacker}"}), 404
    
    else:    #  Both attacker and defender given
        try:
            result = damage_array["attackers"][attacker][defender]
            return jsonify({
                "attacker": attacker,
                "defender": defender,
                "data": result
            })
        except KeyError:
            return jsonify({"error": f"No data for {attacker} vs {defender}"}), 404

In [ ]:
from threading import Thread #Without thread the kernel gets blocked and you cant run further cells.
def run_app():
    app.run(port=5000)

thread = Thread(target=run_app)
thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [ ]:
from flask import render_template_string,request ,jsonify


app = Flask(__name__)

LOCAL_DAMAGE_SERVER = "http://127.0.0.1:5000"  #local server where all the jsons are kept
POKEAPI_BASE = "https://pokeapi.co/api/v2/pokemon/"

HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>Pokemon Type Checker</title>
</head>
<body>
    <h1>Pokemon Type Checker</h1>
    <form method="post" action="/pokemon">
        <label>Enter Pokemon name:</label>
        <input type="text" name="name">
        <input type="submit" value="Search">
    </form>

    {% if result %}
        <h2>Result for {{ result.pokemon }}</h2>
        <p><b>Types:</b> {{ result.types }}</p>
        <h3>Damage Relations:</h3>
        <pre>{{ result.damage_relations | tojson(indent=2) }}</pre>
        {% if result.types|length == 1 %}
            <p><b>Immune against: </b>{{result.immune}} </p>
            <p><b>Weakness against: </b>{{result.weakness}} </p>
        {% elif result.types|length == 2 %}
            <p><b>Immune against: </b>{{result.immune2}} </p>
            <p><b>WeaknessX2 against: </b>{{result.weakness_2}} </p>
            <p><b>WeaknessX4 against: </b>{{result.weakness_4}} </p>
        {% else %}
            <p><b>Types:</b> None found</p>
        {% endif %}
    {% endif %}
    
</body>
</html>
"""

@app.route("/", methods=["GET"])
def index():
    return render_template_string(HTML_TEMPLATE)

@app.route("/pokemon", methods=["POST"])
def get_pokemon():
    pokemon_name = request.form["name"].lower() #lower to avoid ambiguity
    poke_url = f"{POKEAPI_BASE}{pokemon_name}" 
    poke_response = requests.get(poke_url)

    if poke_response.status_code != 200: #in case of error
        return render_template_string(HTML_TEMPLATE, result={"pokemon": pokemon_name, "types": [], "damage_relations": {"error": "Pokémon not found"}})

    poke_data = poke_response.json() #converting to json for easier data handling
    types = [t["type"]["name"] for t in poke_data["types"]] #getting all th types of pokemon

    damage_data = {} 
    for t in types: 
        local_response = requests.get(f"{LOCAL_DAMAGE_SERVER}?defender={t}")#calling our local server for defender attributes
        if local_response.status_code == 200:
            damage_data[t] = local_response.json()
        else:
            damage_data[t] = {"error": "No data available"}

    
    result = {  
        "pokemon": pokemon_name,
        "types": types,
        "damage_relations": damage_data
    }
    weakness=[]
    immune=[]
    overall2={}
    weakness_2=[]
    weakness_4=[]
    immune2=[]
    if len(result["types"])==1: #one type of pokemon
            overall=result["damage_relations"][types[0]]["data"]
            for ele in overall:
                 if overall[ele]==2:
                      weakness.append(ele)
                 elif overall[ele]==0: 
                      immune.append(ele)
            result["immune"]=immune
            result["weakness"]=weakness

    else: #two types of pokemon
            for t in result["damage_relations"][types[0]]["data"]:
                    val1 = result["damage_relations"][types[0]]["data"][t]
                    val2 = result["damage_relations"][types[1]]["data"][t]
                    overall2[t] = val1 * val2
            for ele in overall2:
                if overall2[ele]==2:
                     weakness_2.append(ele)
                elif overall2[ele]==4:
                    weakness_4.append(ele)
                elif overall2[ele]==0:
                     immune2.append(ele)
            result["immune2"]=immune2
            result["weakness_2"]=weakness_2
            result["weakness_4"]=weakness_4
    return render_template_string(HTML_TEMPLATE, result=result)



In [ ]:
def run_app():
    app.run(port=5001)

thread = Thread(target=run_app)
thread.start()

 * Serving Flask app '__main__'
 * Debug mode: off



 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
